In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score, confusion_matrix, classification_report, roc_auc_score, log_loss
import matplotlib.pyplot as plt
import joblib

In [2]:
original_df = pd.read_csv('Data/diabetes_012_health_indicators_BRFSS2015.csv')
# Check missing values
print(original_df.isnull().sum())


Diabetes_012            0
HighBP                  0
HighChol                0
CholCheck               0
BMI                     0
Smoker                  0
Stroke                  0
HeartDiseaseorAttack    0
PhysActivity            0
Fruits                  0
Veggies                 0
HvyAlcoholConsump       0
AnyHealthcare           0
NoDocbcCost             0
GenHlth                 0
MentHlth                0
PhysHlth                0
DiffWalk                0
Sex                     0
Age                     0
Education               0
Income                  0
dtype: int64


In [3]:
print('Distribution for the values of for the target variable\n')
print(original_df["Diabetes_012"].value_counts())

Distribution for the values of for the target variable

Diabetes_012
0.0    213703
2.0     35346
1.0      4631
Name: count, dtype: int64


In [4]:
y = original_df.Diabetes_012
X = original_df.drop('Diabetes_012', axis=1)
X.head(1)

,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,1.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0


In [5]:
# Numerical Predictors
numerical_cols = ['BMI', 'MentHlth', 'PhysHlth', 'Age']
# numerical_cols = list(dict.fromkeys(numerical_cols))  # Remove duplicates just in case

# Categorical Predictors
categorical_cols = [
    'HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 'HeartDiseaseorAttack',
    'PhysActivity', 'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare',
    'NoDocbcCost', 'GenHlth', 'DiffWalk', 'Sex', 'Education', 'Income'
]

# One-hot encode categorical features and drop the first category of each
X_cat = pd.get_dummies(X, columns=categorical_cols, drop_first=True)

# Extract the numerical features
X_num = X[numerical_cols]

# Concatenate numerical and encoded categorical features
X = pd.concat([X_num, X_cat], axis=1)

# Remove any duplicated columns that may result from concat
X = X.loc[:, ~X.columns.duplicated()]

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scaler will only fit on training data so, calculates Mean & STD and stores this on the scaler
scaler = StandardScaler()
X_train[numerical_cols] = scaler.fit_transform(X_train[numerical_cols])
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols]) # Uses scaler that has only seen the traning data -> standardize 

# Combine transformed training group +  assigned clean, matching row indices
df_train = pd.concat([X_train.reset_index(drop=True), y_train.reset_index(drop=True)], axis=1)

In [6]:
# Set a z-score threshold: ±1 SD → ~68.3% of data, ±2 SD → ~95.5% of data, ±3 SD → ~99.7% of data
threshold = 3    

# Calculate absolute z-scores for numerical columns
z_scores = np.abs(df_train[numerical_cols])

# Mask: rows where any numerical feature has a z-score >= threshold of 3
outlier_mask = (z_scores >= threshold).any(axis=1)

# Get the outlier rows
outliers = df_train[outlier_mask]

# Get the cleaned training data (rows that are NOT outliers)
df_train_cleaned = df_train[~outlier_mask].reset_index(drop=True)

# Show the outliers
# print("Outlier rows removed from training data:")
# print(outliers)

# Print how many were removed
print(f"\nRemoved {len(outliers)} outliers from training data.")



Removed 12308 outliers from training data.


In [7]:
# Refit scaler on cleaned training data
scaler = StandardScaler()
df_train_cleaned[numerical_cols] = scaler.fit_transform(df_train_cleaned[numerical_cols])

# Apply same transformation to test data
X_test[numerical_cols] = scaler.transform(X_test[numerical_cols])

# Summary of the original standardized training data (before outlier removal)
print("Original Standardized Training Data:")
print(df_train[numerical_cols].describe())

# Summary of the re-standardized, cleaned training data
print("\nCleaned + Re-standardized Training Data:")
print(df_train_cleaned[numerical_cols].describe())

Original Standardized Training Data:
                BMI      MentHlth      PhysHlth           Age
count  2.029440e+05  2.029440e+05  2.029440e+05  2.029440e+05
mean   2.687503e-16  4.513016e-17 -7.142400e-18 -1.776497e-16
std    1.000002e+00  1.000002e+00  1.000002e+00  1.000002e+00
min   -2.475946e+00 -4.299017e-01 -4.871967e-01 -2.303520e+00
25%   -6.611649e-01 -4.299017e-01 -4.871967e-01 -6.658286e-01
50%   -2.074697e-01 -4.299017e-01 -4.871967e-01 -1.075203e-02
75%    3.974572e-01 -1.606003e-01 -1.431950e-01  6.443245e-01
max    1.052998e+01  3.609619e+00  2.952820e+00  1.626939e+00

Cleaned + Re-standardized Training Data:
                BMI      MentHlth      PhysHlth           Age
count  1.906360e+05  1.906360e+05  1.906360e+05  1.906360e+05
mean   6.336278e-18 -5.529334e-17  4.804389e-17 -2.767463e-17
std    1.000003e+00  1.000003e+00  1.000003e+00  1.000003e+00
min   -2.876142e+00 -4.142922e-01 -4.554221e-01 -2.304848e+00
25%   -7.114224e-01 -4.142922e-01 -4.554221e-01 -6.74

In [8]:
# Separate features and target -> To save after removing outiers and fit + transform
X_train_cleaned = df_train_cleaned.drop(columns='Diabetes_012')
y_train_cleaned = df_train_cleaned['Diabetes_012']

# Save Train
# Save training data
X_train_cleaned.to_csv('Data/X_train_cleaned.csv', index=False)
y_train_cleaned.to_csv('Data/y_train_cleaned.csv', index=False)

# Save Test data
X_test.to_csv(f'Data/X_test_cleaned.csv', index=False)
y_test.to_csv(f'Data/y_test_cleaned.csv', index=False)

print("Cleaned and standardized data saved in 'Data/' folder")

Cleaned and standardized data saved in 'Data/' folder


In [9]:
# Keep only the columns in X_test that are also in X_train_cleaned — and in the same order
X_test = X_test[X_train_cleaned.columns]

In [10]:
from sklearn.multiclass import OneVsRestClassifier
# Train logistic regression
base_logreg = LogisticRegression(max_iter=1000, class_weight='balanced') # assigns larger penalty/weight to mistakes on the minority class
logreg = OneVsRestClassifier(base_logreg)
logreg.fit(X_train_cleaned, y_train_cleaned)

y_pred_orig = logreg.predict(X_test) # → Make training model predictions based on only test data (features only, no target)
# returns a 2D NumPy array with shape (n_samples, n_classes) rows, cols
y_proba_orig = logreg.predict_proba(X_test)[:, 1] # → Probability that the sample belongs to class 1 (positive class)

# --- Evaluation Metrics ---
print("📋 Original Model:")
print(classification_report(y_test, y_pred_orig))

print("\n Confusion Matrix:")
print(confusion_matrix(y_test, y_pred_orig))

📋 Original Model:
              precision    recall  f1-score   support

         0.0       0.95      0.68      0.79     42795
         1.0       0.04      0.22      0.06       944
         2.0       0.33      0.68      0.45      6997

    accuracy                           0.67     50736
   macro avg       0.44      0.53      0.43     50736
weighted avg       0.85      0.67      0.73     50736


 Confusion Matrix:
[[29154  4594  9047]
 [  277   208   459]
 [ 1201  1067  4729]]


In [11]:
logreg_cv = OneVsRestClassifier(
    LogisticRegressionCV(
        Cs=10,
        penalty='l1',
        solver='liblinear',
        cv=5,
        max_iter=1000,
        scoring='roc_auc',
        class_weight='balanced',
        random_state=42
    )
)
logreg_cv.fit(X_train_cleaned, y_train_cleaned)

OneVsRestClassifier(estimator=LogisticRegressionCV(class_weight='balanced',
                                                   cv=5, max_iter=1000,
                                                   penalty='l1',
                                                   random_state=42,
                                                   scoring='roc_auc',
                                                   solver='liblinear'))

In [12]:
for i, estimator in enumerate(logreg_cv.estimators_):
    print(f"Class {i} best C: {estimator.C_[0]}")

Class 0 best C: 0.046415888336127774
Class 1 best C: 2.782559402207126
Class 2 best C: 0.046415888336127774


In [13]:
# Get feature names (used in training)
feature_names = X_train_cleaned.columns

print("\n🧹 Predictors removed (coeff = 0) by L1 Regularization:\n")
for i, estimator in enumerate(logreg_cv.estimators_):
    zero_coef_mask = estimator.coef_[0] == 0
    removed_features = feature_names[zero_coef_mask]

    print(f"Class {i}: {len(removed_features)} predictors removed")
    print(removed_features.tolist())
    print("-" * 50)


🧹 Predictors removed (coeff = 0) by L1 Regularization:

Class 0: 3 predictors removed
['AnyHealthcare_1.0', 'Education_5.0', 'Income_2.0']
--------------------------------------------------
Class 1: 0 predictors removed
[]
--------------------------------------------------
Class 2: 2 predictors removed
['Smoker_1.0', 'Education_3.0']
--------------------------------------------------


In [14]:
joblib.dump(logreg_cv, 'logreg_cv.pkl')
print("Model saved successfully check CWD")

# Predict
y_pred_l1 = logreg_cv.predict(X_test)
y_proba_l1 = logreg_cv.predict_proba(X_test)  # Shape: (n_samples, n_classes)

# Evaluate
print("\n📋 L1-Regularized CV Model (Multiclass):")
print(classification_report(y_test, y_pred_l1))

# Report best C values
print("Best C values per class:")
for i, estimator in enumerate(logreg_cv.estimators_):
    print(f"  Class {i}: Best C = {estimator.C_[0]}")

# Save model
joblib.dump(logreg_cv, 'logreg_cv.pkl')
print("Model saved successfully (logreg_cv.pkl)")

Model saved successfully check CWD

📋 L1-Regularized CV Model (Multiclass):
              precision    recall  f1-score   support

         0.0       0.95      0.68      0.79     42795
         1.0       0.04      0.22      0.06       944
         2.0       0.33      0.67      0.45      6997

    accuracy                           0.67     50736
   macro avg       0.44      0.53      0.43     50736
weighted avg       0.85      0.67      0.73     50736

Best C values per class:
  Class 0: Best C = 0.046415888336127774
  Class 1: Best C = 2.782559402207126
  Class 2: Best C = 0.046415888336127774
Model saved successfully (logreg_cv.pkl)
